In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

c:\Users\Lukas.Schumacher\miniconda3\envs\cogModeling\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [2]:
import sys
sys.path.append("../src")

from macro_models import gaussian_process, batched_gaussian_process
from micro_models import simple_batch_diffusion
from helpers import build_distance_matrix
from priors import diffusion_prior, length_scale_prior, diffusion_prior_gp

In [3]:
%load_ext autoreload
%autoreload 2

In [ ]:
# gpu setting and checking
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
print(tf.config.list_physical_devices('GPU'))

In [4]:
N_SIM = 100
N_OBS = 100
N_SAMPLES = 4000
N_PARAMS = 3

In [ ]:
# load simulated data and true params
sim = pd.read_pickle('../data/sim_data/static_dm_data_100.pkl')
x_nn = sim['rt']
micro_true = sim['theta']
x_nn.shape

## Prepare dynamic stan posteriors

In [5]:
import re
from os import listdir
from os.path import isfile, join

def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]

In [ ]:
mypath = "C:/Users/Lukas.Schumacher/Documents/GitHub/dynamic_dm/stan/dynamic_stan_fits"
files = [f for f in listdir(mypath) if isfile(join(mypath, f))]
files.sort(key=natural_keys)
len(files)

In [ ]:
dynamic_stan_post = np.empty((N_SIM, N_OBS, N_SAMPLES, N_PARAMS))
for i in range(len(files)):
    post_samples = pd.read_csv('dynamic_stan_fits/' + files[i], index_col=False)
    dynamic_stan_post[i] = post_samples.to_numpy()[:, 3:].reshape(100, 4000, 3)

dynamic_stan_post.shape

In [ ]:
# get mean and std of posterior stds
dynamic_stan_post_std_means = dynamic_stan_post.std(axis=2).mean(axis=0)
dynamic_stan_post_std_stds = dynamic_stan_post.std(axis=2).std(axis=0)

In [ ]:
# get posterior means
dynamic_stan_post_means = dynamic_stan_post.mean(axis=2)
dynamic_stan_post_means.shape

In [ ]:
dynamic_stan_abs_error = np.empty((N_SIM, N_OBS, N_PARAMS))
for i in range(N_SIM):
    dynamic_stan_abs_error[i] = np.abs(dynamic_stan_post_means[i] - micro_true[i])
dynamic_stan_abs_error.shape

In [ ]:
dynamic_stan_abs_error_mean = dynamic_stan_abs_error.mean(axis=0)
dynamic_stan_abs_error_std = dynamic_stan_abs_error.std(axis=0)

## Neural Training

In [ ]:
def generator_fun(batch_size, T):
#     macro_prior_mean = beta.mean(1, 25)
#     macro_prior_sd = beta.std(1, 25)
#     micro_prior_means = gamma.mean([2.5, 4, 1.5], scale=[1/2, 1/3, 1/5])
#     micro_prior_sds = gamma.std([2.5, 4, 1.5], scale=[1/2, 1/3, 1/5])

    theta = diffusion_prior(batch_size, n_cond=1)
    eta = random_walk_prior(batch_size, 3)
    theta_t = random_walk(theta, eta, T)
    rt = simple_batch_diffusion(theta_t).astype(np.float32)

#     eta_z = z_standardize(eta, macro_prior_mean, macro_prior_sd)
    
#     theta_t_z = theta_t.copy()
#     for i in range(theta_t.shape[0]):
#         theta_t_z[i] =  z_standardize(theta_t[i], micro_prior_means,  micro_prior_sds)

    return eta.astype(np.float32), theta_t.astype(np.float32), rt

In [199]:
batch_size = 8
T = 100
n_params = 6
dist_mat = build_distance_matrix(T)
length_scales = length_scale_prior(batch_size, 3)
theta0 = diffusion_prior_gp(batch_size, n_cond=1)

In [200]:
%%time
gp_samples = batched_gaussian_process(theta0, dist_mat, length_scales, amplitudes=[0.1, 0.1, 0.03])
rt = simple_batch_diffusion(gp_samples)

Wall time: 140 ms
